In [12]:
from ioh import get_problem, ProblemClass
from ioh import logger
import math
import os
from dataclasses import dataclass
import BA 
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.random_projection import GaussianRandomProjection
import botorch
import gpytorch
from gpytorch.constraints import Interval
from gpytorch.kernels import MaternKernel, ScaleKernel, RBFKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from torch.quasirandom import SobolEngine

from botorch.acquisition.analytic import LogExpectedImprovement
from botorch.exceptions import ModelFittingError
from botorch.fit import fit_gpytorch_mll
from botorch.generation import MaxPosteriorSampling
from botorch.models import SingleTaskGP
from botorch.optim import optimize_acqf
from sklearn.preprocessing import StandardScaler


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {device}")
dtype = torch.float
SMOKE_TEST = os.environ.get("SMOKE_TEST")

# problem dimension
dim = 40 if not SMOKE_TEST else 50
projected_dim = 15

# initial sample points
n_init = 10 if not SMOKE_TEST else 4
max_cholesky_size = float("inf")

problem = get_problem(1, dimension=dim, instance=2, problem_class=ProblemClass.BBOB)

Running on cpu


In [14]:
EVALUATION_BUDGET = 100 if not SMOKE_TEST else 10
NUM_RESTARTS = 3 if not SMOKE_TEST else 2
RAW_SAMPLES = 51 if not SMOKE_TEST else 4
N_CANDIDATES = min(dim, max(200, 20 * dim)) if not SMOKE_TEST else 4

In [15]:
n_sample = 40
X_sample = BA.get_initial_points(10, n_sample)
S = BA.embedding_matrix_pca(input_dim=10, target_dim=3, data=X_sample)
X = torch.randint(100, (7, 2))
print(f"S before increase\n{S}")
print(f"X before increase\n{X}")

S before increase
tensor([[ 0.4321,  0.0342,  0.1614,  0.4346, -0.3942, -0.5921,  0.0643,  0.1235,
         -0.1440,  0.2261],
        [-0.0670,  0.4098, -0.5844, -0.2170, -0.2966,  0.0608,  0.2185, -0.2141,
         -0.2721,  0.4238],
        [ 0.0267, -0.5508, -0.3849, -0.1510, -0.1583, -0.1534, -0.4961, -0.0355,
          0.3634,  0.3113]], dtype=torch.float64)
X before increase
tensor([[87, 83],
        [68, 34],
        [15, 47],
        [63, 22],
        [51, 41],
        [75, 20],
        [72, 27]])


In [16]:
state = BA.BaxusState(dim=dim, eval_budget=EVALUATION_BUDGET
)
n_sample = n_init
X_sample = BA.get_initial_points(state.dim, n_sample)
S = BA.embedding_matrix_pca(input_dim=state.dim, target_dim=state.d_init, data=X_sample)

    # X_baxus_target = BA.get_initial_points(state.d_init, n_init)
X_baxus_target = X_sample @ S.T
X_baxus_target = torch.clamp(X_baxus_target, min=-5, max=5)

    # X_baxus_input = X_baxus_target @ S
X_baxus_input = X_baxus_target@ S

Y_baxus = torch.tensor(
            [-problem(list(x)) for x in X_baxus_input], dtype=dtype, device=device
        ).unsqueeze(-1)
with botorch.settings.validate_input_scaling(False):
    for _ in range(EVALUATION_BUDGET - n_sample):  # Run until evaluation budget depleted
                # Fit a GP model
            train_Y = (Y_baxus - Y_baxus.mean()) / Y_baxus.std()
            likelihood = GaussianLikelihood(noise_constraint=Interval(1e-8, 1e-3))
            model = SingleTaskGP(
                    X_baxus_target, train_Y, likelihood=likelihood
                )
            mll = ExactMarginalLogLikelihood(model.likelihood, model)

                # Fit the model using Cholesky context
            with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                try:
                    fit_gpytorch_mll(mll)
                except ModelFittingError:
                        # Use Adam-based optimization if Cholesky decomposition fails
                    optimizer = torch.optim.Adam([{"params": model.parameters()}], lr=0.1)
                    for _ in range(200):
                        optimizer.zero_grad()
                        output = model(X_baxus_target)
                        loss = -mll(output, train_Y.flatten())
                        loss.backward()
                        optimizer.step()

                    # Generate new candidates
                X_next_target = BA.create_candidate(
                        state=state,
                        model=model,
                        X=X_baxus_target,
                        Y=train_Y,
                        device=device,
                        dtype=dtype,
                        n_candidates=N_CANDIDATES,
                        num_restarts=NUM_RESTARTS,
                        raw_samples=RAW_SAMPLES,
                        acqf="ts",
                    )
                
                # Map new candidates to high-dimensional space
            X_next_input = X_next_target @ S
            Y_next = torch.tensor(
                    [-problem(list(x)) for x in X_next_input], dtype=dtype, device=device
                ).unsqueeze(-1)

                # Update state and concatenate new points
            state = BA.update_state(state=state, Y_next=Y_next)
            X_baxus_input = torch.cat((X_baxus_input, X_next_input), dim=0)
            X_baxus_target = torch.cat((X_baxus_target, X_next_target), dim=0)
            Y_baxus = torch.cat((Y_baxus, Y_next), dim=0)
                
            print(
                    f"iteration {len(X_baxus_input)}, d={len(X_baxus_target.T)})  Best value: {state.best_value:.3}, TR length: {state.length:.3}"
                )

            if state.restart_triggered:
                state.restart_triggered = False
                print("increasing target space")

                S, X_baxus_target = BA.increase_embedding_and_observations(
                        S, X_baxus_target, state.new_bins_on_split
                    )
                print(f"new dimensionality: {len(S)}")
                    
                state.target_dim = len(S)
                state.length = state.length_init
                state.failure_counter = 0
                state.success_counter = 0

InputDataError: Expected all inputs to share the same dtype. Got torch.float64 for X, torch.float32 for Y, and None for Yvar.